In [18]:
# Paso 1: Instalar Dependencias
%pip install backtesting pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Paso 2: Importar las Librerías Necesarias
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
# from backtesting.test import GOOG


In [19]:
import ccxt
ex = ccxt.binance()
from_ts = ex.parse8601('2024-06-17 00:00:00')
symbol = "BTCUSDT"
timeframe= "5m"

ohlcv = ex.fetch_ohlcv(
    symbol, 
    timeframe, 
    since=from_ts, 
    limit=1000)


ExchangeNotAvailable: binance GET https://api.binance.com/api/v3/exchangeInfo 451  {
  "code": 0,
  "msg": "Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error."
}

In [9]:
def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()

# Calcular la Media Móvil Exponencial (EMA)
def EMA(array, n):
    """Simple moving average"""
    return pd.Series(array).ewm(span=n, adjust=False).mean()


def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough

    print("ALERTA",array, n)
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

In [17]:
# importar data 
datapath = '../backtesting/data/BTCUSDT5m.csv'
data = pd.read_csv(datapath, parse_dates=True, index_col=0, )
data = (data / 1e6).assign(Volume=data.Volume * 1e6)  # μBTC OHLC prices


## Estrategias

In [18]:
# Mean Reversion
class MeanReversionStrategy(Strategy):
    def init(self):
        # Inicializa las Bandas de Bollinger
        price = self.data.Close
        print(price)
        self.ma = self.I(SMA, price, 20)  # Media Móvil Simple de 20 períodos
        self.upper_band = self.I(SMA, price, 5)  # Media Móvil Simple de 20 períodos
        self.lower_band = self.I(SMA, price, 25)  # Media Móvil Simple de 20 períodos


        

    def next(self):
        price = self.data.Close[-1]

        # Señal de compra: el precio cruza por debajo de la banda inferior
        if crossover(self.lower_band, price):
            self.buy()

        # Señal de venta: el precio cruza por encima de la banda superior
        elif crossover(price, self.upper_band):
            self.sell()

bt = Backtest(data, MeanReversionStrategy, cash=10000, commission=.002)
stats = bt.run()

stats

[0.07152111 0.07144365 0.07138    ... 0.06659747 0.06660002 0.06662862]


Start                     2024-03-15 00:00:00
End                       2024-06-16 13:05:00
Duration                     93 days 13:05:00
Exposure Time [%]                   99.803471
Equity Final [$]                  9880.120606
Equity Peak [$]                  11937.925018
Return [%]                          -1.198794
Buy & Hold Return [%]               -6.840624
Return (Ann.) [%]                    4.285599
Volatility (Ann.) [%]               78.684456
Sharpe Ratio                         0.054466
Sortino Ratio                        0.088595
Calmar Ratio                         0.160903
Max. Drawdown [%]                  -26.634697
Avg. Drawdown [%]                   -2.939796
Max. Drawdown Duration       45 days 19:55:00
Avg. Drawdown Duration        2 days 12:30:00
# Trades                                   17
Win Rate [%]                        35.294118
Best Trade [%]                       2.094498
Worst Trade [%]                     -17.01592
Avg. Trade [%]                    

In [6]:
# RSI cross strategy
class RSIcross(Strategy):
  rsi_period = 17 
  slPercent = 0.99
 
  def init(self):
    price = self.data.Close
    self.rsi = self.I(RSI,self.data.Close,self.rsi_period)
    # self.rsi = self.I(self.RSI, price, 14)
    # self.rsi = self.I(RSI, pd.Series(self.data.Close), self.rsi_period)

  def next(self):
    price = self.data.Close[-1]

    if self.rsi[-1] < 30:
      self.position.close()

    if self.position.is_long:
      return

    self.buy(sl = price * self.slPercent)

bt = Backtest(data,RSIcross,cash = 100)
stats = bt.run()
stats


ALERTA [0.07152111 0.07144365 0.07138    ... 0.06659747 0.06660002 0.06662862] 17


Start                     2024-03-15 00:00:00
End                       2024-06-16 13:05:00
Duration                     93 days 13:05:00
Exposure Time [%]                   99.988876
Equity Final [$]                     88.12209
Equity Peak [$]                      101.4336
Return [%]                          -11.87791
Buy & Hold Return [%]               -6.840624
Return (Ann.) [%]                  -35.818842
Volatility (Ann.) [%]               40.609635
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -24.985067
Avg. Drawdown [%]                  -24.985067
Max. Drawdown Duration       93 days 12:35:00
Avg. Drawdown Duration       93 days 12:35:00
# Trades                                  392
Win Rate [%]                        30.612245
Best Trade [%]                       9.405781
Worst Trade [%]                     -1.001574
Avg. Trade [%]                    

In [13]:
# momentun

import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover



# Identificar patrones de velas (ejemplo: Engulfing Pattern)
def is_bullish_engulfing(data):
    return (data['Open'].shift(1) > data['Close'].shift(1)) & (data['Close'] > data['Open']) & (data['Close'] > data['Open'].shift(1)) & (data['Open'] < data['Close'].shift(1))

def is_bearish_engulfing(data):
    return (data['Open'].shift(1) < data['Close'].shift(1)) & (data['Close'] < data['Open']) & (data['Close'] < data['Open'].shift(1)) & (data['Open'] > data['Close'].shift(1))


class MomentumTradingStrategy(Strategy):
    def init(self):
        # Inicializar indicadores
        self.sma_short = self.I(SMA, self.data.Close, 20)
        self.sma_long = self.I(SMA, self.data.Close, 50)
        self.rsi = self.I(RSI, self.data.Close, 14)
        # self.bullish_engulfing = self.I(is_bullish_engulfing, self.data)
        # self.bearish_engulfing = self.I(is_bearish_engulfing, self.data)

    def next(self):
        if crossover(self.sma_short, self.sma_long) and self.rsi[-1] > 50 :#and self.bullish_engulfing[-1]:
            self.buy()
        elif crossover(self.sma_long, self.sma_short) and self.rsi[-1] < 50:# and self.bearish_engulfing[-1]:
            self.sell()

# Ejecutar el backtest
bt = Backtest(data, MomentumTradingStrategy, cash=10000, commission=.002)
stats = bt.run()
stats



ALERTA [0.07152111 0.07144365 0.07138    ... 0.06659747 0.06660002 0.06662862] 14


Start                     2024-03-15 00:00:00
End                       2024-06-16 13:05:00
Duration                     93 days 13:05:00
Exposure Time [%]                   99.592109
Equity Final [$]                  9577.804032
Equity Peak [$]                  11561.037835
Return [%]                           -4.22196
Buy & Hold Return [%]               -6.840624
Return (Ann.) [%]                    4.261955
Volatility (Ann.) [%]               78.714426
Sharpe Ratio                         0.054145
Sortino Ratio                        0.088049
Calmar Ratio                         0.160761
Max. Drawdown [%]                    -26.5112
Avg. Drawdown [%]                   -3.004276
Max. Drawdown Duration       45 days 19:55:00
Avg. Drawdown Duration        2 days 15:50:00
# Trades                                   12
Win Rate [%]                        33.333333
Best Trade [%]                      12.816323
Worst Trade [%]                     -16.48854
Avg. Trade [%]                    